In [110]:
import numpy as np
import pandas as pd
import pyaudio
import wave
import librosa
from scipy.io.wavfile import write
from keras.models import model_from_json
import pickle

In [111]:
json_file = open('models/40/final_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights('models/40/final_model.h5')

In [128]:
# Define constants for audio recording
format = pyaudio.paInt16
channels = 1
filename = "test/happy.wav"
duration=5
sample_rate=44100
chunk_size=1024
duration=30

In [129]:
# Define function to record audio
def record_audio():
    audio = pyaudio.PyAudio()
    stream = audio.open(format=format, channels=channels,
                        rate=sample_rate, input=True,
                        frames_per_buffer=chunk_size)
    print("Recording...")
    frames = []

    # Calculate total number of chunks
    total_chunks = int(sample_rate / chunk_size * duration)
    chunks_per_marker = total_chunks // 10  # Number of chunks for each '#' character

    for i in range(total_chunks):
        data = stream.read(chunk_size)
        frames.append(data)

    print("\nFinished recording.")

    stream.stop_stream()
    stream.close()
    audio.terminate()

    wavefile = wave.open(filename, 'wb')
    wavefile.setnchannels(channels)
    wavefile.setsampwidth(audio.get_sample_size(format))
    wavefile.setframerate(sample_rate)
    wavefile.writeframes(b''.join(frames))
    wavefile.close()

In [130]:
# Define constants for audio feature extraction
mfcc_sample_rate = 22050
n_mfcc = 40
axis_mfcc = 1

In [131]:
def segment_audio(segment_length,sr,y):

    # Calculate the length of each segment in samples
    segment_samples = int(segment_length * sr)

    # Calculate the total number of segments
    num_segments = len(y) // segment_samples

    segments = []

    # Extract segments
    for i in range(num_segments):
        start_sample = i * segment_samples
        end_sample = (i + 1) * segment_samples
        segment = y[start_sample:end_sample]
        segments.append(segment)

    return segments

In [132]:
def preprocess_audio(filename, sample_rate=22050, n_mfcc=40, offset_s=0.5):
    # Load audio file
    y, sr = librosa.load(filename, sr=sample_rate, mono=True, offset=offset_s)
    y_trimmed, _ = librosa.effects.trim(y)

    return segment_audio(segment_length=5,sr=sr,y=y_trimmed)

In [133]:
#record_audio()

In [134]:
segments = preprocess_audio(filename)

In [135]:
segments

[array([-3.6446705e-05, -5.0137372e-05, -2.6092066e-05, ...,
         1.8418679e-02,  1.7382352e-02,  1.5644714e-02], dtype=float32),
 array([ 0.01273728,  0.00885533,  0.00448044, ..., -0.01707608,
        -0.01268003, -0.01094848], dtype=float32),
 array([-0.01006793, -0.00881656, -0.00694277, ...,  0.0110437 ,
         0.01377037,  0.01493229], dtype=float32),
 array([0.01519687, 0.01360911, 0.01059486, ..., 0.01880057, 0.01818263,
        0.01840654], dtype=float32),
 array([ 0.01856009,  0.01891422,  0.01993833, ..., -0.03093199,
        -0.0287811 , -0.02659699], dtype=float32)]

In [57]:
ravdess = pd.DataFrame(columns=['feature'])
bookmark=0
for i in segments:
    mfccs = librosa.feature.mfcc(y=i, sr=sr, n_mfcc=n_mfcc)
    mfccs_mean = np.mean(mfccs, axis=1)
    #Add MFCCs feature results to list
    ravdess.loc[bookmark] = [mfccs_mean]
    bookmark=bookmark+1

In [64]:
ravdess

,feature
0,"[-319.9953, 169.50264, -6.675172, 8.762605, -1..."
1,"[-367.64487, 156.09085, -23.756754, 36.41025, ..."
2,"[-373.31763, 148.85207, -14.554767, 35.97845, ..."
3,"[-370.79465, 138.36104, -15.291364, 33.030426,..."
4,"[-380.7688, 150.3276, -25.56467, 40.309135, 2...."


In [68]:
#Turn array into dataframe
ravdess_final = pd.DataFrame(ravdess['feature'].values.tolist())

In [80]:
#Analyze new dataframe shape
ravdess_final.shape

(5, 40)

In [81]:
with open('meta/scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

In [82]:
x_cnn = scaler.transform(ravdess_final)

In [87]:
x = np.expand_dims(x_cnn, axis=2)

In [89]:
x.shape

(5, 40, 1)

In [95]:
predictions = np.argmax(model.predict(x),axis=-1)

In [100]:
max(predictions)

5

In [101]:
emotions = {0:'neutral', 1:'calm', 2:'happy', 3:'sad', 4:'angry', 5:'fear', 6:'disgust', 7:'surprised'}

In [103]:
emotions[max(predictions)]

'fear'